In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from typing import Type

from collections import  

In [ ]:
#CLASSES
class MandyocModel:
    def __init__(self, model_name, layers: list, Nx, Nz, Lx, Lz):
        """"
        This class creates a model with the given properties
        model_name: str
            Name of the model
        layers: list 
            List containing MandyocLayer objects
        Nx: int
            Number of points in x direction
        Nz: int
            Number of points in z direction
        Lx: float
            Length of the model in x direction [m]
        Lz: float
            Length of the model in z direction [m]
        """

        self.model_name = model_name
        self.layers = layers
        self.Nx = Nx
        self.Nz = Nz
        self.Lx = Lx
        self.Lz = Lz

        self.dx = Lx/Nx
        self.dz = Lz/Nz

        self.x = np.linspace(0, Lx, Nx)
        self.z = np.linspace(Lz, 0, Nz)

        self.X, self.Z = np.meshgrid(self.x, self.z)

        for layer in layers:
            setattr(self, layer.layer_label, layer)

class MandyocLayer:
    def __init__(self, layer_label, rheology: type,
                 density, effective_viscosity_scale_factor=1.0, radiogenic_heat_production=0.0,
                 base_depth=0.0e3, Nx=None):
        """"
        This class creates a layer with the given properties
        layer_label: str
            Name of the layer
        rheology: class
            Rheological properties of the layer onbtained from LithologicalUnit class
        density: float
            Density of the layer [kg/m3]
        interface: NoneType or np.array
            Interface of the layer.
            If None, the interface will be setted after the creation of the Layer.
            If np.array, the interface is defined by the given array.
        effective_viscosity_scale_factor: float
            Scale factor for the effective vistocisty
        radiogenic_heat_production: float
            Radiogenic heat production of the layer [W/kg]
        base_depth: float
            Depth of the layer base [m]
        Nx: int
            Number of points in x direction
        """

        self.layer_label = layer_label
        self.rheology = LithologicalUnit(rheology)
        self.density = density
        self.effective_viscosity_scale_factor = effective_viscosity_scale_factor
        self.radiogenic_heat_production = radiogenic_heat_production
        self.rheology_name = self.rheology.name
        self.pre_exponential_constant = self.rheology.pre_exponential_constant
        self.power_law_exponent = self.rheology.power_law_exponent
        self.activation_energy = self.rheology.activation_energy
        self.activation_volume = self.rheology.activation_volume
        self.base_depth = base_depth
        self.Nx = Nx

        self.interface = np.ones(Nx)*base_depth

class LithologicalUnit:
    """"
    This class calls the respective rheological properties of the given mineral

    mineral_name: class
        Mineral rheology written in CamelCase. For example, WetOlivine, DryOlivine, WetQuartz
    """
    def __init__(self, mineral_name: type):
        self.mineral_name = mineral_name() # mineral_name is a class, so we need to call it to get the object
        self.name = self.mineral_name.name
        self.pre_exponential_constant = self.mineral_name.pre_exponential_constant
        self.power_law_exponent = self.mineral_name.power_law_exponent
        self.activation_energy = self.mineral_name.activation_energy
        self.activation_volume = self.mineral_name.activation_volume

class WetOlivine:
    """
    Wet olivine rheological properties
    """
    def __init__(self):
        self.name = 'wet_olivine'
        self.pre_exponential_constant = 1.393e-14
        self.power_law_exponent = 3
        self.activation_energy = 429.0e3
        self.activation_volume = 15.0e-6

class DryOlivine:
    """
    Dry olivine rheological properties
    """
    def __init__(self):
        self.name = 'dry_olivine'
        self.pre_exponential_constant = 2.4168e-15
        self.power_law_exponent = 3.5
        self.activation_energy = 540.0e3
        self.activation_volume = 25.0e-6

class WetQuartz:
    """
    Wet quartz rheological properties
    """
    def __init__(self):
        self.name = 'wet_quartz'
        self.pre_exponential_constant = 8.574e-28
        self.power_law_exponent = 4.0
        self.activation_energy = 222.0e3
        self.activation_volume = 0.0

class Air:
    """
    Air rheological properties
    """
    def __init__(self):
        self.name = 'air'
        self.pre_exponential_constant = 1.0e-18
        self.power_law_exponent = 1.0
        self.activation_energy = 0.0
        self.activation_volume = 0.0


In [14]:
Nx = 1001
Nz = 301
Lx = 1000.0e3
Lz = 300.0e3

# x = np.linspace(0, Lx, Nx)
# z = np.linspace(Lz, 0, Nz)

# X, Z = np.meshgrid(x, z)

# thickness of sticky air layer (m)
thickness_air = 40 * 1.0e3
# thickness of upper crust (m)
thickness_upper_crust = 20 * 1.0e3
# thickness of lower crust (m)
thickness_lower_crust = 15 * 1.0e3
# thickness of lithospheric mantle
thickness_lithospheric_mantle = 45 * 1.0e3

# total thickness of lithospheric mantle (m)
thickness_litho = 80 * 1.0e3
# thickness_litho = 120 * 1.0e3

asthenosphere = MandyocLayer('asthenosphere', WetOlivine,
                            density=3300.0,
                            effective_viscosity_scale_factor=1.0,
                            radiogenic_heat_production=7.38e-12)

lithospheric_mantle = MandyocLayer('lithospheric_mantle', DryOlivine,
                                    density=3354.0,
                                    # interface=np.ones(Nx) * (thickness_litho + thickness_air),
                                    effective_viscosity_scale_factor=1.0,
                                    radiogenic_heat_production=9.0e-12,
                                    base_depth=thickness_air+thickness_upper_crust+thickness_lower_crust+thickness_lithospheric_mantle,
                                    Nx=Nx)

lower_crust = MandyocLayer('lower_crust', WetQuartz,
                            density=2800.0,
                            # interface=np.ones(Nx) * (thickness_lower_crust + thickness_upper_crust + thickness_air),
                            effective_viscosity_scale_factor=1.0,
                            radiogenic_heat_production=2.86e-10,
                            base_depth=thickness_air+thickness_upper_crust+thickness_lower_crust,
                            Nx=Nx) #0.8e-6 / 2800.0)

upper_crust = MandyocLayer('upper_crust', WetQuartz,
                            density=2800.0,
                            # interface=np.ones(Nx) * (thickness_upper_crust + thickness_air),
                            effective_viscosity_scale_factor=1.0,
                            radiogenic_heat_production=9.26e-10,
                            base_depth=thickness_air+thickness_upper_crust,
                            Nx=Nx)#2.5e-6 / 2700.0)

air = MandyocLayer('air', Air,
                    density=1.0,
                    # interface=np.ones(Nx) * (thickness_air),
                    effective_viscosity_scale_factor=1.0,
                    radiogenic_heat_production=0.0,
                    base_depth=thickness_air,
                    Nx=Nx)

layers = [asthenosphere, lithospheric_mantle, lower_crust, upper_crust, air]

model = MandyocModel('rifting', layers, Nx, Nz, Lx, Lz)

In [23]:
model.asthenosphere.effective_viscosity_scale_factor

1.0

In [13]:
rho_lc = 2800 #lower crust density [kg/m3]
C_lc = 1.0 #lower crust scaling factor
H_lc = 2.5e-6 / 2700.0 #lower crust radiogenic heat production [W/kg]

Nx = 1001
thickness_lower_crust = 40.0
lc_interface = np.ones(Nx) * thickness_lower_crust

lower_crust = MandyocLayer(WetOlivine, rho_lc, lc_interface, C_lc, H_lc)


In [14]:
asthenosphere.interface = 12

In [15]:
asthenosphere.interface

12

array([40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40.])

8.574e-28

_tuplegetter(0, 'Alias for field number 0')

8.574e-28
4.0
